In [1]:
# Import dependencies
import subprocess
import json
import os
from dotenv import load_dotenv
from bit.network import NetworkAPI

In [2]:
from web3 import Web3
from dotenv import load_dotenv
from web3.middleware import geth_poa_middleware
from eth_account import Account
from bit import PrivateKeyTestnet



In [3]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)


In [4]:
# Load and set environment variables
load_dotenv()
mnemonic = os.getenv("mnemonic")

In [5]:
# Import constants.py and necessary functions from bit and web3
# YOUR CODE HERE
# think that I need to define mneumonic from a path variable
# from constants import *
# look up the guidence regsrding store

In [6]:
def derive_wallets(coin):
    command = f'php derive -g --mnemonic="{mnemonic}" --cols=path,address,privkey,pubkey --format=json --coin="{coin}" --numderive=3'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [7]:
derive_wallets("btc-test")

[{'path': "m/44'/1'/0'/0/0",
  'address': 'n4Fp14Cauo9PB9TTMo8v9sR5hpALPWxZo8',
  'privkey': 'cP5GnCwRxkZi5ofnV1eaAXChSzzZtLFxNgTS4ErdJR26S47YVCYQ',
  'pubkey': '0202b4b456432a3443b39745207684452ab6c228976cf4e73f871a3d93eddd1683'},
 {'path': "m/44'/1'/0'/0/1",
  'address': 'mqS7Wm4o73LQeNz6YeZvPN4PLtmuJmtTv5',
  'privkey': 'cR6Up7cDWc6RRe7VKvqu4Fs3f88LoaAFm2D9d1pgywSmHf76DEcU',
  'pubkey': '0367a297d7e64346e6c06b8bab6a099ec2ede6b069475e235bf5e35a95234076bf'},
 {'path': "m/44'/1'/0'/0/2",
  'address': 'n1dxYj2ma6L8MjehfaJ7Zgb9tn8QzcpTNW',
  'privkey': 'cTT2oXX3WWESkFS77qCtyyVtskCyPTyNmSpvQZcoVqBpQeVeBaQG',
  'pubkey': '02859d6f50b9aa7b8a3f46089431f75603f146abaee51a7648ea3188af4350fc52'}]

In [8]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {
    "ETH":derive_wallets("ETH"),
    "BTC":derive_wallets("BTC"),
    "btc-test":derive_wallets("btc-test")
    
}

In [9]:
coins

{'ETH': [{'path': "m/44'/60'/0'/0/0",
   'address': '0xE2C082d87260E01723e045A3ed16673f7Bd5887c',
   'privkey': '0xdbe47b703e8f215211b15111991911606b2f0df7af4a6ffd3578dc33d5e53fec',
   'pubkey': '0252771d6faf99634dc170cd68c49e21c5352fbcf51d61b8538f4bb1fc19b05a6c'},
  {'path': "m/44'/60'/0'/0/1",
   'address': '0x0dcf835C586E46450283B903F278F754e6f8B55D',
   'privkey': '0x1939a361d691f5cc80162aa3f1d10e55ef6ad076bebc1785f7077bfeb91f87b1',
   'pubkey': '0257d88d99bb82e5a43885428cf092300d1c6f99196b2ffcd2ff496701e20f2f38'},
  {'path': "m/44'/60'/0'/0/2",
   'address': '0x501d2F04e286E75E15B686F657B5455570613320',
   'privkey': '0xec7f11cd379753eaf8c37d900bcaf5c62009b8b2af4669ebc3b4b6f99cc03aca',
   'pubkey': '02a066cfe66fc05bec7be823a88dae4c384db0c56fcdee750365889f5652117eb5'}],
 'BTC': [{'path': "m/44'/0'/0'/0/0",
   'address': '1L7SzesPSCkxTNr3CY8QgcRU74XLhJ8zS2',
   'privkey': 'KxejuJiJLJ6Cz7ivmP47peCCQdpdSonNuNxApKqxHoG5fH3AAaBH',
   'pubkey': '0288c6cff4917579778c7d40d4bddb45c9c4f4f7aa

In [10]:
eth_pkey = coins["ETH"][0]['privkey']

btest_pkey = coins["btc-test"][0]['privkey']

In [11]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    print(priv_key)
    if coin == "ETH":
        return Account.privateKeyToAccount(priv_key)
    elif coin == "btc-test":
        return PrivateKeyTestnet(priv_key)


In [12]:
BTC = "btc"

In [13]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin,account, recipient, amount):
    if coin == "ETH": 
        gasEstimate = w3.eth.estimateGas(
            {"from":account.address, "to":recipient, "value": amount}
        )
        return { 
            "from": account.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(eth_acc.address)
        }
    
    elif coin == "btc-test":
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])


In [27]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_txn(coin,account,recipient, amount):
    if coin == "ETH":
        tx = create_tx(coin, account, recipient, amount)
        signed_tx = account.sign_transaction(tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        print(result.hex())
        return result.hex()
    elif coin == "btc-test":
        tx = create_tx(coin, account, recipient, amount)
        tx_btctest = create_tx(coin, account, recipient, amount)
        signed_txn = account.sign_transaction(tx)
        print(signed_tx)
        return NetworkAPI.broadcast_tx_testnet(signed_tx)

In [15]:
#BTC Test
account = priv_key_to_account("btc-test",btest_pkey)


cP5GnCwRxkZi5ofnV1eaAXChSzzZtLFxNgTS4ErdJR26S47YVCYQ


In [16]:
unsigned_tx = create_tx("btc-test",account,"n4Fp14Cauo9PB9TTMo8v9sR5hpALPWxZo8",0.001)

In [17]:
send = send_txn("btc-test",account,"n4Fp14Cauo9PB9TTMo8v9sR5hpALPWxZo8",.0001)

010000000279dafd39e75889eb42d800d232f29a41cf84b3aa3cbbc6059f13399d0354f08b010000006a4730440220229fa12b0c980f3a5e19ca57ff1ba58bb9d7e870c2b945fea41d7dd4b769615402202c365f10a33c3669d3e55fc20c2ec4fada4ff838a940d4760b1d52f8f1f965f501210202b4b456432a3443b39745207684452ab6c228976cf4e73f871a3d93eddd1683ffffffff79dafd39e75889eb42d800d232f29a41cf84b3aa3cbbc6059f13399d0354f08b000000006a4730440220057ba3a0e4b0b841db0b9fb0c077d00c82641d86c9ca3b335c9a97513756513f022021463eff3a16c0df49324f23a31da7dd8afe2a778ed804e811413547b5dac15801210202b4b456432a3443b39745207684452ab6c228976cf4e73f871a3d93eddd1683ffffffff0210270000000000001976a914f96dbb1be16ae85c6af4bddba79c5dc53a6f60f988ac8f8d1100000000001976a914f96dbb1be16ae85c6af4bddba79c5dc53a6f60f988ac00000000


In [18]:
# https://tbtc.bitaps.com/8bf054039d39139f05c6bb3caab384cf419af232d200d842eb8958e739fdda79/n4Fp14Cauo9PB9TTMo8v9sR5hpALPWxZo8

In [28]:
# ETC Test
account_2 = priv_key_to_account("ETH",eth_pkey)

0xdbe47b703e8f215211b15111991911606b2f0df7af4a6ffd3578dc33d5e53fec


In [ ]:
send = send_txn("ETH",account_2,"0x687422eea2cb73b5d3e242ba5456b782919afc85",.0001)